## Introduction to Databases
Relational Databases only
MySQL, Postgres, SQLite

- query languages happen in both relational and nonrelational databases

### Relational databases
- predefined relationships between contents
- organized into tables with cols and rows in the tidy direction
    - rows are records and should be related with cols the features/type
- multiple tables often exist in on DB
    - connect tables using a primary key: unique identifier that connects rows amidst disparate tables

#### Creating and updating
- updating requires "Relational Database Management System" RDMS
- these us SQL language
    - syntax changes depending which type of database we're using


#### SQL statement
- SELECT\*FROM 
    - retrieves information from the data
    
### Non-relational databases
- doesn't use tables/rows
- storage may be in trees, links, JSON docs, dictionaries, graphs, etc. 
- this handles unstructured, diverse data
- 4 types
    - document stores like JSON or xml data
    - each piece of data is a document that is unique and changes over time
    - columnar databases
        - DB organized by columns where the rows aren't related, so each row can have a different number/type of cols
    - graph db
        - edges are relationships between the 
- popular databases
    - mongDB
    - Amazon DynamoDB
        - can't query data by anything other than the key, but one value can have more than one key
    - Cassandra
    - Neo4j
- programming foundations: databases cover non-relational sets

### Accessing DB in Python
- modules may have different interactions
- python created specs for uses: Python Database API
    - allows code to be transferrable across db
    - then we can use the same module style across all the python modules

### Connecting to DB
- we need a connect function that returns a connect object
    - we can close, commit, rollback any transaction
        - all or nothing changes in db
- create cursor object from the connection
    - allows us to work within the db and fetch, execute, etc. 
    - arrisize attribute that specifies the number of rows to fetch (defaults to 1)
- each db module has constructors for the datatypes
- when data passed to cursor methods for execution, type is detected and bound

## Using SQLite in Python

- light in terms of setup and resources
- doesn't require server to run
- databases are local with files stored on disks
- self-contained system
- transaction: set of queries we want to take place at once with ACID compliance
- good way for protoyping that we can port to larger db later

    

In [2]:
import sqlite3

connection = sqlite3.connect('movies.db') # where to save the database or open if exists
cursor = connection.cursor() # create cursor
cursor.execute('''CREATE TABLE IF NOT EXISTS Movies (Title TEXT, Director TEXT, Year INT)''') 

In [3]:
connection.commit()
connection.close()

In [5]:
connection = sqlite3.connect('movies.db') # where to save the database
cursor = connection.cursor()

# Create 
cursor.execute("INSERT INTO Movies VALUES ('Taxi Driver', 'Martin Scorsese', 1976)") #if we do this more than once adds more than once!!

# retrieve
cursor.execute("SELECT * FROM Movies")
print(cursor.fetchone()) #grab one item and print it

('Taxi Driver', 'Martin Scorsese', 1976)


In [7]:
famousFilms = [('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012)]
cursor.executemany('Insert INTO Movies VALUES (?,?,?)', famousFilms)

#retreive
# makes another object so we can iterate twice, first for the fechall and then for the print statement 
records = cursor.execute("SELECT * FROM Movies") 
print(cursor.fetchall())

for record in records: 
    print(record)

[('Taxi Driver', 'Martin Scorsese', 1976), ('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012), ('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012)]


In [10]:
release_year = (1985,)
cursor.execute("SELECT * FROM Movies WHERE year=?", release_year)
# using string makes the SQL insecure!!!! that's why we aren't doing it

print(cursor.fetchall())

[('Back to the Future', 'Steven Spielberg', 1985), ('Back to the Future', 'Steven Spielberg', 1985)]


In [17]:
def delete_all_records(conn, loc):
    """
    Delete all rows in the table
    :param conn: connection to SQLite database
    :loc: the location to delete from
    :return:
    """
    sql = "DELETE FROM " +loc
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

In [20]:
delete_all_records(connection, 'MOVIES')

# now add our movies back in as I'd doubled them
cursor.executemany('Insert INTO Movies VALUES (?,?,?)', famousFilms)`

In [21]:
connection.commit()
connection.close()

### SQLAlchemy
 - Allow us to work with our language of choice so we don't have to use the SQL language within our language
 - SQLAlchemy Core 
     - schhema centric view
     - uses SQL expression language
 - and SQLAlchemy ORM
     - object centric view 
     - maintains the consistency between the language of choice and the SQL
     - can also use the SQL expression language during it
     - pros: abstracts DB system simplifying interaction for python developers
     - cons: less performant than SQL for advanced SQL users and can prevent from understanding the SQL under teh hood
- engine allows multiple db connections and manages them itself
    - create with sqlalchemy.create_engine and save as variable (hertofor referred to as engine)
    - by running engine.connect() from an engine object,  we create a proxy to the python API of the database connected to the engine
    - sqlalchemy.MetaData will retrieve all our metadata about our connection
    - when we select something from our talbe through the connection, returns a result proxy
        - we can use this proxy to retrieve the data
            - fetchall() will return a result set object
            - this is the actual data and we can interact with it as an object
                - however, changes to this list are NOT connected to our db as it's not a query -- this is a new python object

 

In [23]:
# import the data from movies db
import sqlalchemy as db

engine = db.create_engine('sqlite:///movies.db')

In [38]:
connection = engine.connect()

metadata = db.MetaData()
movies = db.Table('Movies', metadata, autoload=True, autoload_with=engine)

In [39]:
# select all records from teh moves table

query = db.select([movies]) # by making an object doesn't yet run it

result_proxy = connection.execute(query)

In [40]:
result_set = result_proxy.fetchall()
print(result_set[0])

('Pulp Fiction', 'Quentin Tarantino', 1994)


In [41]:
print(result_proxy, result_set) # does changing the result set change the proxy data? 

<sqlalchemy.engine.cursor.LegacyCursorResult object at 0x1143fb190> [('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012)]


In [46]:
result_set[0] = ('p', 'nope', 1994)

# grab all again to print

query = db.select([movies])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set)

# Doesn't change the database!!

[('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012), ('Psycho', 'Alfred Hitchcock', 1960)]


In [44]:
query = db.select([movies]).where(movies.columns.Director=="Wes Anderson")
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set)

[('Moonrise Kingdom', 'Wes Anderson', 2012)]


In [45]:
# insert a new record

query = movies.insert().values(Title='Psycho', Director='Alfred Hitchcock', Year=1960)
connection.execute(query)

# grab all again to print

query = db.select([movies])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set)

[('Pulp Fiction', 'Quentin Tarantino', 1994), ('Back to the Future', 'Steven Spielberg', 1985), ('Moonrise Kingdom', 'Wes Anderson', 2012), ('Psycho', 'Alfred Hitchcock', 1960)]
